In [2]:
!pip install kaggle --quiet

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/keys/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split, Dataset
from PIL import Image
import time
# Configurar dispositivo (GPU si está disponible)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda:0


In [6]:
# --- Celda 2: Subir tu Framework de Ataque ---
!mkdir -p attacks
!mkdir -p models

print("\nPor favor, sube tus archivos .py del framework usando el panel de la izquierda.")
print("Sube a 'attacks/': base.py, carlini_wagner.py")
print("Sube a 'models/': base_models.py, pytorch.py")
print("Sube al directorio raíz: adversarial.py, criteria.py, distances.py, utils.py")


Por favor, sube tus archivos .py del framework usando el panel de la izquierda.
Sube a 'attacks/': base.py, carlini_wagner.py
Sube a 'models/': base_models.py, pytorch.py
Sube al directorio raíz: adversarial.py, criteria.py, distances.py, utils.py


In [7]:
import os
from google.colab import files

In [12]:
# --- Celda 3: Descarga y Preparación del Dataset (CORREGIDA) ---
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Dataset
from PIL import Image
import os

# Descargar y descomprimir el dataset
!kaggle competitions download -c sci-pediatric-chest-x-ray-pneumonia-classificatio --force -q
!unzip -oq "sci-pediatric-chest-x-ray-pneumonia-classificatio.zip"

# --- RUTAS CORRECTAS ---
train_dir = 'train/train'  # Apunta al directorio que contiene las carpetas NORMAL/PNEUMONIA
test_dir = 'test/test'      # Apunta al directorio con todas las imágenes de prueba juntas

print(f"Directorio de entrenamiento usado: {train_dir}")
print(f"Directorio de prueba usado: {test_dir}")

# --- Transformaciones de Imagen para ResNet ---
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3), # ResNet espera 3 canales
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val_test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# --- LÓGICA DE SPLIT (imitando a TensorFlow) ---
full_train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['val_test'])
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])
train_dataset.dataset.transform = data_transforms['train'] # Aplicar aumentación solo a train

# --- CLASE DATASET PERSONALIZADA PARA EL TEST SET (sin carpetas de clase) ---
class UnlabeledImageFolder(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = sorted([f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f))])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        # Devolvemos un label ficticio (ej. -1) porque el dataloader lo requiere, pero no lo usaremos
        return image, -1

test_dataset = UnlabeledImageFolder(test_dir, transform=data_transforms['val_test'])

# --- CREAR DATALOADERS ---
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2),
    'test': DataLoader(test_dataset, batch_size=1, shuffle=False) # No barajar para mantener el orden
}

class_names = full_train_dataset.classes
print(f"\nClases encontradas: {class_names}")
print(f"Tamaño del set de entrenamiento: {len(train_dataset)}")
print(f"Tamaño del set de validación: {len(val_dataset)}")
print(f"Tamaño del set de prueba: {len(test_dataset)}")

Directorio de entrenamiento usado: train/train
Directorio de prueba usado: test/test

Clases encontradas: ['NORMAL', 'PNEUMONIA']
Tamaño del set de entrenamiento: 3052
Tamaño del set de validación: 764
Tamaño del set de prueba: 2040


In [14]:
# --- Transformaciones de Imagen para ResNet ---
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3), # ResNet espera 3 canales
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val_test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# --- LÓGICA DE SPLIT ---
full_train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['val_test'])
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])
train_dataset.dataset.transform = data_transforms['train']

# --- CLASE DATASET PERSONALIZADA PARA EL TEST SET ---
class UnlabeledImageFolder(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = sorted([f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f))])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        # Devolvemos un label ficticio (ej. -1), no lo usaremos
        return image, -1

test_dataset = UnlabeledImageFolder(test_dir, transform=data_transforms['val_test'])

# --- CREAR DATALOADERS Y DICCIONARIO DE DATASETS ---
image_datasets = {'train': train_dataset, 'val': val_dataset, 'test': test_dataset}
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2),
    'test': DataLoader(test_dataset, batch_size=1, shuffle=False)
}

class_names = full_train_dataset.classes
print(f"\nClases encontradas: {class_names}")
print(f"Tamaño del set de entrenamiento: {len(train_dataset)}")
print(f"Tamaño del set de validación: {len(val_dataset)}")
print(f"Tamaño del set de prueba: {len(test_dataset)}")


Clases encontradas: ['NORMAL', 'PNEUMONIA']
Tamaño del set de entrenamiento: 3052
Tamaño del set de validación: 764
Tamaño del set de prueba: 2040


In [15]:
# --- Celda 4: Entrenamiento de la ResNet18 ---
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import time
import copy

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --- Bucle de Entrenamiento ---
num_epochs = 5
best_acc = 0.0
model_save_path = "resnet18_xray_best.pth"

print("\nIniciando entrenamiento...")
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}'); print('-' * 10)
    for phase in ['train', 'val']:
        model.train() if phase == 'train' else model.eval()
        running_loss, running_corrects = 0.0, 0
        dataset_size = len(image_datasets[phase]) # Esta línea ahora funcionará

        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                if phase == 'train':
                    loss.backward(); optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_size
        epoch_acc = running_corrects.double() / dataset_size
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            torch.save(model.state_dict(), model_save_path)
            print(f"--> Nuevo mejor modelo guardado en '{model_save_path}'")

print(f"\nEntrenamiento finalizado. Mejor precisión de validación: {best_acc:.4f}")


Iniciando entrenamiento...
Epoch 1/5
----------
train Loss: 0.2166 Acc: 0.9246
val Loss: 0.2562 Acc: 0.9215
--> Nuevo mejor modelo guardado en 'resnet18_xray_best.pth'
Epoch 2/5
----------
train Loss: 0.1194 Acc: 0.9551
val Loss: 0.1095 Acc: 0.9607
--> Nuevo mejor modelo guardado en 'resnet18_xray_best.pth'
Epoch 3/5
----------
train Loss: 0.1206 Acc: 0.9535
val Loss: 0.1940 Acc: 0.9136
Epoch 4/5
----------
train Loss: 0.0884 Acc: 0.9676
val Loss: 0.1053 Acc: 0.9634
--> Nuevo mejor modelo guardado en 'resnet18_xray_best.pth'
Epoch 5/5
----------
train Loss: 0.0770 Acc: 0.9695
val Loss: 0.1471 Acc: 0.9359

Entrenamiento finalizado. Mejor precisión de validación: 0.9634


In [ ]:
# --- Celda 5: Generación de Ejemplos Adversariales ---

# Añadir el directorio actual al path de Python para que encuentre tus módulos
import sys
sys.path.insert(0, os.getcwd())

from models.pytorch import PyTorchModel
from attacks.carlini_wagner import CarliniWagnerL2Attack
from criteria import Misclassification
from distances import MeanSquaredDistance as MSE
from adversarial import Adversarial
from tqdm.notebook import tqdm

# Cargar el mejor modelo entrenado
model.load_state_dict(torch.load(model_save_path))
model.to(device); model.eval()

# Wrappear el modelo para el framework
mean = np.array([0.485, 0.456, 0.406]); std = np.array([0.229, 0.224, 0.225])
min_val, max_val = np.min((0 - mean) / std), np.max((1 - mean) / std)
bounds = (min_val, max_val)

fmodel = PyTorchModel(model, bounds=bounds, num_classes=len(class_names), channel_axis=1, device=device)

# --- Bucle de Generación ---
print("\nIniciando la generación de ejemplos adversariales...")
benign_images, adv_images, adv_distances = [], [], []
N_SAMPLES = 30
attack = CarliniWagnerL2Attack()
generated_count = 0

# Usamos tqdm para una barra de progreso
for i, (data, _) in enumerate(tqdm(dataloaders['test'], total=N_SAMPLES, desc="Generando Adversariales")):
    if generated_count >= N_SAMPLES:
        break

    image_np = data.squeeze(0).permute(1, 2, 0).cpu().numpy()

    # Predecir la clase para usarla como etiqueta a atacar
    original_pred = np.argmax(fmodel.predictions(image_np))

    # Crear el objeto adversarial
    adversarial = Adversarial(fmodel, Misclassification(), image_np, original_pred, distance=MSE)

    # Ejecutar el ataque con los hiperparámetros optimizados
    attack(adversarial, binary_search_steps=5, max_iterations=100, learning_rate=1e-2)

    if adversarial.image is not None:
        benign_images.append(image_np)
        adv_images.append(adversarial.image)
        adv_distances.append(adversarial.distance.value)
        generated_count += 1

print(f"\n\nProceso finalizado. Se generaron {len(benign_images)} pares de imágenes.")


Iniciando la generación de ejemplos adversariales...


Generando Adversariales:   0%|          | 0/30 [00:00<?, ?it/s]


Búsqueda binaria paso 1/5, const = 1.0e-03
  Iteración 100/100, Pérdida: 0.0017, Mejor L2: 0.1248
  Éxito para const=1.0e-03. Nuevo límite superior: 1.0e-03

Búsqueda binaria paso 2/5, const = 5.0e-04
  Iteración 100/100, Pérdida: 0.0016, Mejor L2: 0.1220
  Éxito para const=5.0e-04. Nuevo límite superior: 5.0e-04

Búsqueda binaria paso 3/5, const = 2.5e-04
  Iteración 100/100, Pérdida: 0.0015, Mejor L2: 0.1496
  Éxito para const=2.5e-04. Nuevo límite superior: 2.5e-04

Búsqueda binaria paso 4/5, const = 1.3e-04
  Iteración 100/100, Pérdida: 0.0015, Mejor L2: 0.1444
  Éxito para const=1.3e-04. Nuevo límite superior: 1.3e-04

Búsqueda binaria paso 5/5, const = 6.3e-05
  Iteración 100/100, Pérdida: 0.0015, Mejor L2: 0.2346
  Éxito para const=6.3e-05. Nuevo límite superior: 6.3e-05

Búsqueda binaria paso 1/5, const = 1.0e-03
  Iteración 100/100, Pérdida: 0.0041, Mejor L2: 1.7603
  Éxito para const=1.0e-03. Nuevo límite superior: 1.0e-03

Búsqueda binaria paso 2/5, const = 5.0e-04
  Iterac

In [ ]:
# --- Celda 6: Guardar y Descargar los Resultados (CORREGIDA) ---

import shutil # Importamos la librería shutil

# Montar Google Drive si no está montado
from google.colab import drive
if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')

save_dir = '/content/drive/MyDrive/XRay_Adversarial_Data/'
os.makedirs(save_dir, exist_ok=True)

if 'benign_images' in locals() and benign_images:
    # Mover el modelo guardado a Drive usando shutil.move
    if os.path.exists(model_save_path):
        shutil.move(model_save_path, os.path.join(save_dir, model_save_path))

    # Guardar los arrays de imágenes y distancias
    np.save(os.path.join(save_dir, "benign_images_xray.npy"), np.array(benign_images))
    np.save(os.path.join(save_dir, "adv_images_xray.npy"), np.array(adv_images))
    np.save(os.path.join(save_dir, "adv_distances_xray.npy"), np.array(adv_distances))

    print("\nArchivos guardados en tu Google Drive en la carpeta 'XRay_Adversarial_Data/':")
    print(f"- {model_save_path}")
    print("- benign_images_xray.npy")
    print("- adv_images_xray.npy")
    print("- adv_distances_xray.npy")

    print("\n¡Ya puedes descargar esta carpeta desde Google Drive a tu PC local para el análisis!")
else:
    print("No se generaron imágenes adversariales, por lo que no se guardó nada.")


Archivos guardados en tu Google Drive en la carpeta 'XRay_Adversarial_Data/':
- resnet18_xray_best.pth
- benign_images_xray.npy
- adv_images_xray.npy
- adv_distances_xray.npy

¡Ya puedes descargar esta carpeta desde Google Drive a tu PC local para el análisis!
